In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchmetrics.detection import IntersectionOverUnion
from PIL import Image
import os
import cv2
import numpy as np
import json
import shutil
from torchsummary import summary
import csv

In [2]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("icaslab/dataset-face-detection-for-edge-computing-class")
print("Path to dataset files:", path)

# Download latest version
path = kagglehub.dataset_download("reganwillis/testset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dataset-face-detection-for-edge-computing-class
Path to dataset files: /kaggle/input/testset


In [4]:
#Data base addresses
base_adress = '/kaggle/input/dataset-face-detection-for-edge-computing-class/Dataset_FDDB/Dataset_FDDB/images'
labels_adr = '/kaggle/input/dataset-face-detection-for-edge-computing-class/Dataset_FDDB/Dataset_FDDB/label.txt'

In [5]:
# Make the labels ready

with open(labels_adr, 'r') as f:
    lines = f.readlines()
annotations = []
bboxes = []
flag = False
for line in lines:
    if line.startswith('#'):
      if flag:
        annotations.append({'image':img_name, 'bboxes': bboxes})
        bboxes = []
      flag = True
      img_name = line[2:]
    else:
      x_min, y_min, x_max, y_max = line.split()
      bboxes.append([int(x_min), int(y_min), int(x_max), int(y_max)])

In [6]:
# Custom Dataset Class for FDDB
class FDDBDataset(Dataset):
    def __init__(self, img_dir, annot_file, target_size=(224, 224), transform=None):
        self.img_dir = img_dir
        self.target_size = target_size
        self.transform = transform
        self.data = self._parse_annotations(annot_file)

    def _parse_annotations(self, annot_file):
        
        data = []
        for el in annot_file:
          img_path = os.path.join(self.img_dir, el['image'][:-1])
          boxes = el['bboxes']
          data.append((img_path, boxes))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, boxes = self.data[idx]
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")

        # Original dimensions
        h, w, _ = image.shape

        # Resize image
        image_resized = cv2.resize(image, self.target_size)
        target_h, target_w = self.target_size

        # Scale bounding boxes
        scale_x = target_w / w
        scale_y = target_h / h
        boxes_resized = []
        for box in boxes:
            x_min = int(box[0] * scale_x)
            y_min = int(box[1] * scale_y)
            x_max = int(box[2] * scale_x)
            y_max = int(box[3] * scale_y)
            boxes_resized.append([x_min, y_min, x_max, y_max])

        # Convert to tensor
        if self.transform:
            image_resized = self.transform(image_resized)
        else:
            image_resized = transforms.ToTensor()(image_resized)

        return image_resized, torch.tensor(boxes_resized, dtype=torch.float32)

In [7]:
# DataLoader preparation
def get_dataloaders(img_dir, annot_file, batch_size=16, target_size=(224, 224), validation_split=0.2):

    # Transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Dataset
    dataset = FDDBDataset(img_dir, annot_file, target_size, transform)

    # Split dataset
    val_size = int(len(dataset) * validation_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    return train_loader, val_loader

def collate_fn(batch):
    """
    Custom collate function to handle variable-length bounding box arrays.

    :param batch: List of tuples (image, boxes).
    :return: Tuple of images and targets.
    """
    images = torch.stack([item[0] for item in batch])
    targets = [item[1] for item in batch]
    return images, targets

In [8]:
class MobileNetFaceDetector(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=2,
            padding=1
        )
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU6()

        # bottleneck
        #hidden_dim = 32
        self.bottleneck1 = nn.Sequential(
            #nn.Conv2d(in_channels=32, out_channels=32, kernel_size=1, stride=1, padding=0, bias=False),
            #nn.BatchNorm2d(32),
            #nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1, groups=32, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(16),
        )
        #hidden_dim = 16*6
        self.bottleneck2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(16*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=16*6, out_channels=16*6, kernel_size=3, stride=2, padding=1, groups=16*6, bias=False),
            nn.BatchNorm2d(16*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=16*6, out_channels=24, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(24),
        )
        self.bottleneck21 = nn.Sequential(
            nn.Conv2d(in_channels=24, out_channels=24*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(24*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=24*6, out_channels=24*6, kernel_size=3, stride=1, padding=1, groups=24*6, bias=False),
            nn.BatchNorm2d(24*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=24*6, out_channels=24, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(24),
        )
        #hidden_dim = 24*6
        self.bottleneck3 = nn.Sequential(
            nn.Conv2d(in_channels=24, out_channels=24*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(24*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=24*6, out_channels=24*6, kernel_size=3, stride=2, padding=1, groups=24*6, bias=False),
            nn.BatchNorm2d(24*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=24*6, out_channels=32, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(32),
        )
        self.bottleneck31 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(32*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=32*6, out_channels=32*6, kernel_size=3, stride=1, padding=1, groups=32*6, bias=False),
            nn.BatchNorm2d(32*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=32*6, out_channels=32, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(32),
        )
        self.bottleneck32 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(32*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=32*6, out_channels=32*6, kernel_size=3, stride=1, padding=1, groups=32*6, bias=False),
            nn.BatchNorm2d(32*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=32*6, out_channels=32, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(32),
        )
        #hidden_dim=32*6
        self.bottleneck4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(32*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=32*6, out_channels=32*6, kernel_size=3, stride=2, padding=1, groups=32*6, bias=False),
            nn.BatchNorm2d(32*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=32*6, out_channels=64, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64),
        )
        self.bottleneck41 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=64*6, out_channels=64*6, kernel_size=3, stride=1, padding=1, groups=64*6, bias=False),
            nn.BatchNorm2d(64*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=64*6, out_channels=64, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64),
        )
        self.bottleneck42 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=64*6, out_channels=64*6, kernel_size=3, stride=1, padding=1, groups=64*6, bias=False),
            nn.BatchNorm2d(64*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=64*6, out_channels=64, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64),
        )
        self.bottleneck43 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=64*6, out_channels=64*6, kernel_size=3, stride=1, padding=1, groups=64*6, bias=False),
            nn.BatchNorm2d(64*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=64*6, out_channels=64, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64),
        )
        #hidden_dim=64*6
        self.bottleneck5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=64*6, out_channels=64*6, kernel_size=3, stride=1, padding=1, groups=64*6, bias=False),
            nn.BatchNorm2d(64*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=64*6, out_channels=96, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(96),
        )
        self.bottleneck51 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=96*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(96*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=96*6, out_channels=96*6, kernel_size=3, stride=1, padding=1, groups=96*6, bias=False),
            nn.BatchNorm2d(96*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=96*6, out_channels=96, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(96),
        )
        self.bottleneck52 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=96*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(96*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=96*6, out_channels=96*6, kernel_size=3, stride=1, padding=1, groups=96*6, bias=False),
            nn.BatchNorm2d(96*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=96*6, out_channels=96, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(96),
        )
        #hidden_dim=96*6
        self.bottleneck6 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=96*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(96*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=96*6, out_channels=96*6, kernel_size=3, stride=2, padding=1, groups=96*6, bias=False),
            nn.BatchNorm2d(96*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=96*6, out_channels=160, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(160),
        )
        self.bottleneck61 = nn.Sequential(
            nn.Conv2d(in_channels=160, out_channels=160*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(160*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=160*6, out_channels=160*6, kernel_size=3, stride=1, padding=1, groups=160*6, bias=False),
            nn.BatchNorm2d(160*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=160*6, out_channels=160, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(160),
        )
        self.bottleneck62 = nn.Sequential(
            nn.Conv2d(in_channels=160, out_channels=160*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(160*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=160*6, out_channels=160*6, kernel_size=3, stride=1, padding=1, groups=160*6, bias=False),
            nn.BatchNorm2d(160*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=160*6, out_channels=160, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(160),
        )
        #hidden_dim=160*6
        self.bottleneck7 = nn.Sequential(
            nn.Conv2d(in_channels=160, out_channels=160*6, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(160*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=160*6, out_channels=160*6, kernel_size=3, stride=1, padding=1, groups=160*6, bias=False),
            nn.BatchNorm2d(160*6),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels=160*6, out_channels=320, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(320),
        )

        self.conv2 = nn.Conv2d(
            in_channels=320,
            out_channels=1280,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(1280)
        self.relu2 = nn.ReLU6()

        self.pool = nn.AdaptiveAvgPool2d(1)

        # Custom head for bounding box and classification
        self.fc_bbox = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(512, 4),  # Bounding box: [x_min, y_min, x_max, y_max]
        )
        self.fc_label = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Linear(512, 1),  # Binary classification: face/no face
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.bottleneck1(x)
        x = self.bottleneck2(x)
        x = self.bottleneck21(x)
        x = self.bottleneck3(x)
        x = self.bottleneck31(x)
        x = self.bottleneck32(x)
        x = self.bottleneck4(x)
        x = self.bottleneck41(x)
        x = self.bottleneck42(x)
        x = self.bottleneck43(x)
        x = self.bottleneck5(x)
        x = self.bottleneck51(x)
        x = self.bottleneck52(x)
        x = self.bottleneck6(x)
        x = self.bottleneck61(x)
        x = self.bottleneck62(x)
        x = self.bottleneck7(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.pool(x).view(x.size(0), -1)
        bbox = self.fc_bbox(x)
        label = self.fc_label(x)
        return bbox, label
mobilenetfacedetector = MobileNetFaceDetector().to(device)
summary(mobilenetfacedetector, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             896
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
            Conv2d-9         [-1, 96, 112, 112]           1,536
      BatchNorm2d-10         [-1, 96, 112, 112]             192
            ReLU6-11         [-1, 96, 112, 112]               0
           Conv2d-12           [-1, 96, 56, 56]             864
      BatchNorm2d-13           [-1, 96, 56, 56]             192
            ReLU6-14           [-1, 96,

In [9]:
# Loss functions
bbox_loss_fn = nn.SmoothL1Loss()  # For bounding box regression
label_loss_fn = nn.BCELoss()      # For binary classification

In [10]:
# Training loop
def train_model(train_loader, val_loader, num_epochs, model, arch, learning_rate, batch_size):
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')  # Initialize best validation loss
    best_model_path = f"{arch}_{learning_rate}_{batch_size}.pth"  # Path to save the best model
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for images, targets in train_loader:
            images = images.to(device)
            bboxes = [torch.tensor(t, dtype=torch.float32).to(device) for t in targets]  # List of bounding boxes
            labels = [int(1) for t in targets]  # List of labels
            labels = torch.tensor(labels, dtype=torch.float32).to(device)
            preds_bbox, preds_label = model(images)
            # Compute losses
            bbox_losses = []
            label_losses = []
            for i in range(len(bboxes)):
              bbox_losses.append(bbox_loss_fn(preds_bbox[i], bboxes[i]))
              label_losses.append(label_loss_fn(preds_label[i], labels[i].unsqueeze(-1)))

            bbox_loss = torch.mean(torch.stack(bbox_losses))
            label_loss = torch.mean(torch.stack(label_losses))
            loss = bbox_loss + label_loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

        # Validate and save the best model
        val_loss = validate_model(model, val_loader)
        if val_loss < best_val_loss:
            print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving model...")
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)

    print("Training complete. Best model saved as:", best_model_path)
    return best_val_loss.item()


In [11]:
def normalize_boxes(preds):
    """
    Normalize the 'boxes' in the predictions to ensure they are all tensors of shape [N, 4].
    Args:
        preds: List of dictionaries with 'boxes' and 'labels'.
    Returns:
        Normalized predictions with 'boxes' as tensors of shape [N, 4].
    """
    for pred in preds:
        # If boxes is a list of tensors, stack them into a single tensor
        if isinstance(pred['boxes'], list):
            pred['boxes'] = torch.stack(pred['boxes'])  # Stack into [N, 4]
    return preds

In [12]:
def calculate_iou(pred_box, gt_box):
    """
    Calculate IoU (Intersection over Union) for a single pair of boxes.
    Args:
        pred_box: Tensor of shape (4,), [x_min, y_min, x_max, y_max].
        gt_box: Tensor of shape (4,), [x_min, y_min, x_max, y_max].
    Returns:
        IoU value (float).
    """
    # Determine the (x, y)-coordinates of the intersection rectangle
    x1 = max(pred_box[0], gt_box[0])
    y1 = max(pred_box[1], gt_box[1])
    x2 = min(pred_box[2], gt_box[2])
    y2 = min(pred_box[3], gt_box[3])

    # Compute the area of intersection rectangle
    inter_width = max(0, x2 - x1)
    inter_height = max(0, y2 - y1)
    inter_area = inter_width * inter_height

    # Compute the area of both the predicted and ground-truth rectangles
    pred_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
    gt_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])

    # Compute the area of union
    union_area = pred_area + gt_area - inter_area

    # Compute IoU
    iou = inter_area / union_area if union_area > 0 else 0.0
    return iou

In [13]:
def compute_map(pred_boxes, gt_boxes, threshold=0.5):
    tp = 0
    fp = 0

    # format
    pred_boxes = pred_boxes[0]["boxes"].cpu().detach().numpy().tolist()
    gt_boxes = gt_boxes[0]["boxes"].cpu().detach().numpy().tolist()

    # if there is more than one ground truth box
    # loop over ious for all boxes and pick the
    # closest one to measure mAP
    closest_iou = 0.0
    for i in range(len(gt_boxes)):
        iou = calculate_iou(pred_boxes[0], gt_boxes[i])
        if iou > closest_iou:
            closest_iou = iou
    if closest_iou >= threshold:
        tp += 1
    else:
        fp += 1
    precision = tp / (tp + fp + 1e-6)

    return precision * 100

In [14]:
def validate_model(model, val_loader):
    metric = IntersectionOverUnion().to(device)
    model.eval()
    total_bbox_loss = 0
    total_label_loss = 0
    total_iou = []
    total_map = []
    with torch.no_grad():
        for images, targets in val_loader:
            images = images.to(device)
            bboxes = [torch.tensor(t, dtype=torch.float32).to(device) for t in targets]  # List of bounding boxes
            labels = [int(1) for t in targets]  # List of labels
            labels = torch.tensor(labels, dtype=torch.float32).to(device)
            preds_bbox, preds_label = model(images)

            bbox_losses = []
            label_losses = []

            for i in range(len(bboxes)):
                bbox_losses.append(bbox_loss_fn(preds_bbox[i], bboxes[i]))
                label_losses.append(label_loss_fn(preds_label[i], labels[i].unsqueeze(-1)))
                
                preds = [{"boxes": [preds_bbox[i]], "labels": preds_label[i]}]
                preds = normalize_boxes(preds)
                
                targets_combined = torch.cat([bboxes[i]], dim=0)
                targets = [{"boxes": targets_combined, "labels": torch.ones(len(targets_combined)).to(device)}]

                iou_value = metric(preds, targets)
                total_iou.append(iou_value['iou'].item())
                map_value = compute_map(preds, targets)
                total_map.append(map_value)
            total_bbox_loss += torch.mean(torch.stack(bbox_losses))
            total_label_loss += torch.mean(torch.stack(label_losses))

    # Calculate average validation loss
    avg_bbox_loss = total_bbox_loss / len(val_loader)
    avg_label_loss = total_label_loss / len(val_loader)
    val_loss = avg_bbox_loss + avg_label_loss
    print('IoU = ', sum(total_iou)/len(total_iou))
    print('mAP@50 = ', sum(total_map)/len(total_map))
    print(f"Validation - BBox Loss: {avg_bbox_loss:.4f}, Label Loss: {avg_label_loss:.4f}, Total Loss: {val_loss:.4f}")
    return val_loss

In [15]:
!pip install hyperopt
from hyperopt import fmin, tpe, hp, anneal, Trials

In [16]:
#batch_size = 16
#target_size = (224, 224)
#train_loader, val_loader = get_dataloaders(base_adress, annotations, batch_size, target_size)
#train_model(mobilenetfacedetector, train_loader, val_loader, num_epochs=2)

hyperopt_out = [['score', 'arch', 'learning_rate', 'batch_size']]

def min_func(params):
    params = {'model': params['model'], 'arch': params['arch'],
              'learning_rate': float(params['learning_rate']),
              'batch_size': int(params['batch_size'])}
              #'dropout': float(params['dropout'])}
    target_size = (224, 224)
    train_loader, val_loader = get_dataloaders(base_adress, annotations, params['batch_size'], target_size)
    score = train_model(train_loader, val_loader, num_epochs=40, **params)
    hyperopt_out.append([score, params['arch'],
                         params['learning_rate'], params['batch_size']])

    return score

curr_model = MobileNetFaceDetector().to(device)
mobilenetv2space={'model': curr_model,
                  'arch': 'mobilenetv2',
                  #'learning_rate': hp.loguniform('learning_rate', np.log(1e-5), np.log(1e-2)),
                  'learning_rate': hp.choice('learning_rate', [1e-3]),
                  'batch_size': hp.choice('batch_size', [16, 32, 64])}
                  #'dropout' hp.loguniform('dropout', np.log(0.3), np.log(0.5))}

best_params = fmin(fn=min_func, space=mobilenetv2space, algo=tpe.suggest, max_evals=3)

with open('hyperopt_out.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(hyperopt_out)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

<ipython-input-10-592d80072b0a>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bboxes = [torch.tensor(t, dtype=torch.float32).to(device) for t in targets]  # List of bounding boxes

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([1, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([3, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.sm

Epoch 1/40, Loss: 43.1630
  0%|          | 0/3 [00:24<?, ?trial/s, best loss=?]

<ipython-input-14-3e8f1a4f874e>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bboxes = [torch.tensor(t, dtype=torch.float32).to(device) for t in targets]  # List of bounding boxes



IoU = 
0.24438445957667768
mAP@50 = 
17.13779204948711
Validation - BBox Loss: 34.9344, Label Loss: 0.0000, Total Loss: 34.9344
Validation loss improved from inf to 34.9344. Saving model...
Epoch 2/40, Loss: 27.4809
IoU = 
0.29983391446495833
mAP@50 = 
24.204922791543698
Validation - BBox Loss: 32.7128, Label Loss: 0.0000, Total Loss: 32.7128
Validation loss improved from 34.9344 to 32.7128. Saving model...
Epoch 3/40, Loss: 25.2306
IoU = 
0.36569963863231375
mAP@50 = 
39.929288692619615
Validation - BBox Loss: 26.6573, Label Loss: 0.0000, Total Loss: 26.6573
Validation loss improved from 32.7128 to 26.6573. Saving model...
Epoch 4/40, Loss: 24.1844
IoU = 
0.35334895800489785
mAP@50 = 
35.86568851593707
Validation - BBox Loss: 27.5797, Label Loss: 0.0000, Total Loss: 27.5797
Epoch 5/40, Loss: 23.1848
IoU = 
0.3919967374297076
mAP@50 = 
47.34977597177903
Validation - BBox Loss: 24.2118, Label Loss: 0.0000, Total Loss: 24.2118
Validation loss improved from 26.6573 to 24.2118. Saving mode

In [17]:
class FDDBTestDataset(Dataset):
    def __init__(self, img_dir, target_size=(224, 224)):
        self.img_dir = img_dir
        self.target_size = target_size
        self.transform = transforms.Compose([
                                            transforms.ToTensor(),
                                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                            ])
        self.image_files = []

        for img in os.listdir(img_dir):
            self.image_files.append(img)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.img_dir + '/' + self.image_files[idx]
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        image_resized = cv2.resize(image, self.target_size)
        image_resized = self.transform(image_resized)

        return image_resized, self.image_files[idx]

# DataLoader preparation
def get_test_dataloader(img_dir, batch_size=1, target_size=(224, 224)):
    dataset = FDDBTestDataset(img_dir, target_size)
    test_loader = DataLoader(dataset, batch_size=batch_size)#, collate_fn=collate_fn)

    return test_loader

In [18]:
def inference(model, test_loader, device):
    predictions = []

    model.eval()
    with torch.no_grad():
        for i, (image, image_name) in enumerate(test_loader):
            image = image.to(device)

            # get prediction
            pbbox, plabel = model(image)
            
            # format prediction
            preds = [{"id": image_name, "boxes": [pbbox], "labels": plabel}]
            preds = normalize_boxes(preds)
            predictions.append(preds)
    return predictions

In [19]:
# evaluate - submission

# test dataloader
testset_file_path = '/kaggle/input/testset/testset'
test_dataloader = get_test_dataloader(testset_file_path)

# model inference
for model_path in os.listdir('/kaggle/working'):
    full_path = model_path.split('.')
    model_id = ''.join(full_path[0:len(full_path)-1])
    if full_path[-1] == 'pth':
        print('inferencing on model:', model_id)
        inf_model = MobileNetFaceDetector()
        inf_model.load_state_dict(torch.load(model_path))
        inf_model.to(device)
        predictions = inference(inf_model, test_dataloader, device)
        
        # create submission
        data = [['image_id', 'x1', 'y1', 'x2', 'y2']]
        
        for i in range(len(predictions)):
            img_id = predictions[i][0]['id'][0]
            image = cv2.imread(testset_file_path + '/' + img_id)
            pbbox = predictions[i][0]['boxes'].cpu().tolist()[0][0]
        
            # scale predictions
            h, w = (224, 224)
            target_h, target_w, _ = image.shape
            scale_x = target_w / w
            scale_y = target_h / h
            x_min = int(pbbox[0] * scale_x)
            y_min = int(pbbox[1] * scale_y)
            x_max = int(pbbox[2] * scale_x)
            y_max = int(pbbox[3] * scale_y)
            pbbox = [x_min, y_min, x_max, y_max]
            formatted_img_id = "\'" + img_id + "\'"
            data.append([formatted_img_id, pbbox[0], pbbox[1], pbbox[2], pbbox[3]])
        with open(f'od_out_{model_id}.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(data)

inferencing on model: mobilenetv2_0001_16


<ipython-input-19-d503d9a2e139>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inf_model.load_state_dict(torch.load(model_path))


inferencing on model: mobilenetv2_0001_32


In [20]:
!pip install onnx
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.4/705.4 kB 14.4 MB/s eta 0:00:00


In [21]:
# Model class must be defined somewhere
for PATH in os.listdir('/kaggle/working'):
    full_path = PATH.split('.')
    model_id = ''.join(full_path[0:len(full_path)-1])
    if full_path[-1] == 'pth':
        print('deploying to onnx:', model_id)
        model = MobileNetFaceDetector().to(device)
        model.load_state_dict(torch.load(PATH))
        model.eval()
        dummy_input = torch.randn(1, 3, 224, 224).to(device)  # Adjust shape based on your model's input size
        
        # Export the model to ONNX
        torch.onnx.export(
            model,  # The loaded PyTorch model
            dummy_input,  # Example input tensor
            "model.onnx",  # Output ONNX file name
            export_params=True,  # Store trained parameters
            opset_version=13,  # ONNX version (adjust as needed)
            do_constant_folding=True,  # Optimize by folding constants
            input_names=["input"],  # Naming input tensor
            output_names=["output"],  # Naming output tensor
            dynamic_axes=None 
        )
        
        print("...Model successfully exported to ONNX!")

deploying to onnx: mobilenetv2_0001_16


<ipython-input-21-d3eba58bdb50>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(PATH))


...Model successfully exported to ONNX!
deploying to onnx: mobilenetv2_0001_32
...Model successfully exported to ONNX!
